In [22]:
#Importing all the necessary libraries
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
nltk.download("stopwords")
import re
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
import os
import glob
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
# Extracting all the files.
all_files=os.listdir('/content/drive/MyDrive/IR assignment/Humor,Hist,Media,Food')


  


In [25]:
# Creating a dictionary which contains all file names and their respective contents.
file_content_dict={'filename':[],'content':[]}
for i in all_files:
  file1 = open("/content/drive/MyDrive/IR assignment/Humor,Hist,Media,Food/"+i,"r",encoding='utf-8',errors='ignore')
  file_content_dict['filename'].append(i)
 # text = file1.read().decode(errors='replace')
  text=file1.read()
  file_content_dict['content'].append(text)
  


In [26]:
# Creating a dataframe which consists two columns i.e., filename and content
file_content=pd.DataFrame(file_content_dict)

In [27]:
file_content

,filename,content
0,apsnet.txt,"Date: Wed, 28 Dec 2005 08:16:23 -0700\nFrom: R..."
1,applepie.des,Grandma Hauf's Apple Pie\nCategory: Desserts: ...
2,anthropo.stu,From duncan@rmy.emory.edu Wed Sep 12 10:22:08 ...
3,apsaucke.des,APPLESAUCE SPICE CAKE\nCategory: Desserts: Ca...
4,antibiot.txt,PROPHYLACTICANTIBIOTICENDOCARDITISMEDICALBACTE...
...,...,...
1128,woods.txt,\n C.V. Woods World Champions...
1129,woodsmok.txt,"""Reno, Nevada. Hazardous smog levels yester..."
1130,wood,**** The Woody Sketch\t\t\t\t\t\t\t ****\n*...
1131,worldend.hum,The World Ended Yesterday\n=== ===== ===== ===...


# **Preprocessing**

In [52]:
# Applying tokenization on text corpus.
w_token = nltk.tokenize.WhitespaceTokenizer()

In [53]:
# Tokenization function
def tokenized_text(text):
    return w_token.tokenize(text)

In [54]:
#Converted all the words in lower case.
file_content['content']=file_content['content'].str.lower()
#Removed all the special characters and punctuation marks.
file_content['content'] = file_content['content'].str.replace('[^a-zA-Z0-9]',' ')
file_content['content'] = file_content['content'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
stop_word = stopwords.words('english')
#Removed all the stopwords from the corpus using NLTK stopwords.
file_content['content'] = file_content['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_word)]))
#Performed tokenization on the document corpus.
#Performed lemmatization on the tokens of the corpus.
file_content['content_tokens']=file_content.content.apply(tokenized_text)

In [55]:
file_content

,filename,content,content_tokens
0,apsnet.txt,date wed 28 dec 2005 08 16 23 0700 russ dale j...,"[date, wed, 28, dec, 2005, 08, 16, 23, 0700, r..."
1,applepie.des,grandma hauf apple pie category desserts cake ...,"[grandma, hauf, apple, pie, category, desserts..."
2,anthropo.stu,duncan rmy emory edu wed sep 12 10 22 08 1990 ...,"[duncan, rmy, emory, edu, wed, sep, 12, 10, 22..."
3,apsaucke.des,applesauce spice cake category desserts cake c...,"[applesauce, spice, cake, category, desserts, ..."
4,antibiot.txt,prophylacticantibioticendocarditismedicalbacte...,[prophylacticantibioticendocarditismedicalbact...
...,...,...,...
1128,woods.txt,c v woods world championship chili 1 3 lb chic...,"[c, v, woods, world, championship, chili, 1, 3..."
1129,woodsmok.txt,reno nevada hazardous smog levels yesterday fo...,"[reno, nevada, hazardous, smog, levels, yester..."
1130,wood,woody sketch monty python flying circus transc...,"[woody, sketch, monty, python, flying, circus,..."
1131,worldend.hum,world ended yesterday l tyler c 1985 author ri...,"[world, ended, yesterday, l, tyler, c, 1985, a..."


In [56]:
#Creating unique tokens list and token frequency dictionary.
token_frequency={}
# token_frequency stores the frequency of tokens of the document corpus.
unique_tokens=[]
# unique_tokens stores the list of unique tokes of the document corpus. 
for tokens in file_content['content_tokens']:
  for word in tokens:
          if(word in token_frequency.keys()):
              token_frequency[word] = token_frequency[word] + 1
          else:
              token_frequency[word] = 1
              unique_tokens.append(word)

In [57]:
# Creating posting list for each term
posting_list={}
# posting_list contains the list of documents to which particular token belongs to.
for word in unique_tokens:
    posting_list[word] = []

for i in range(len(file_content)):
    tokens = file_content.iloc[i,2]
    for word in tokens:
        if(i not in posting_list[word]):
            posting_list[word].append(i) 


# **Handling the queries**

In [58]:
# The AND operator function takes 2 posting lists as an argument.
# These posting lists are already sorted in ascending order of doc ids.
# We have used 2 pointer approach to intersect both the posting lists. If we found the equal doc ids, we append it in the result, otherwise, we increment the pointer of smaller doc id posting list. 
# While we are comparing, we have also maintained one variable for the comparison where we increment the variable as we go on comparing the posting list.

def AND_operation(posting_List1, posting_List2): 
  comparison = 0
  results=[]
  i=0
  j=0
  while(i<len(posting_List1) and j<len(posting_List2)): 
    if(posting_List1[i]<posting_List2[j]): 
        comparison+=1
        i+=1
    else:
      if(posting_List1[i]==posting_List2[j]): 
        comparison+=1
        results.append(posting_List1[i]) 
        i+=1
        j+=1
      else:
        comparison+=1
        j+=1
  return results, comparison

In [59]:
# The OR operator function takes 2 posting lists as an argument.
# These posting lists are already sorted in ascending order of doc ids.
# We have used 2 pointer approach to find the union of both the posting lists. If we found the equal doc ids, we append it in the result, otherwise, we append and increment the pointer of smaller doc id posting list. 
# While we are comparing, we have also maintained one variable for the comparison where we increment the variable as we go on comparing the posting list.

def OR_operation(posting_List1, posting_List2): 
  comparison = 0
  results=[]
  i=0
  j=0
  while(i<len(posting_List1) and j<len(posting_List2)): 
    # posting_List1[i] = int(posting_List1[i])
    # posting_List2[j] = int(posting_List2[j])
    if(posting_List1[i]<posting_List2[j]): 
        comparison+=1
        results.append(posting_List1[i]) 
        i+=1
    else:
      if(posting_List1[i]==posting_List2[j]): 
        comparison+=1
        results.append(posting_List1[i]) 
        i+=1
        j+=1
      else:
        comparison+=1
        results.append(posting_List2[j]) 
        j+=1
  return results, comparison

In [60]:
# The NOT operator function takes 1 posting list as an argument.
# The posting list is already sorted in ascending order of doc ids.
# We have found the difference between the list that contains all doc ids and this posting list and then return the result.

def NOT_operation(posting_List):
    indexes = []
    results =[]
    for i in range(len(all_files)): # Creates a temporary result list containing all the indexes
        indexes.append(i)
    main_index =0
    posting_index =0
    for i in range(posting_List[len(posting_List)-1]):
        # print(cur_index_main)
        if(indexes[main_index]==posting_List[posting_index] ):
            posting_index += 1
            main_index += 1
        else:
            results.append(indexes[main_index])
            main_index +=1
    results = results+indexes[main_index+1:]
    return results

In [61]:
# The AND NOT operator function takes 2 posting lists A and B as an argument.
# These posting lists are already sorted in ascending order of doc ids.
# First, we calculate the NOT of posting list B from the NOT operation function and store the immediate result.
# After that, we perform the AND operation between posting list A and the immediate NOT result of posting list B.

def AND_NOT_operation(posting_List1,posting_List2):

  posting_List3 = NOT_operation(posting_List2) 
  results,comparison = AND_operation(posting_List1, posting_List3) 
  return results,comparison





In [62]:
# The OR NOT operator function takes 2 posting lists A and B as an argument.
# These posting lists are already sorted in ascending order of doc ids.
# First, we calculate the NOT of posting list B from the NOT operation function and store the immediate result.
# After that, we perform the OR operation between posting list A and the immediate NOT result of posting list B.

def OR_NOT_operation(posting_List1, posting_List2):

    posting_List3 = NOT_operation(posting_List2)
    results,comparison = OR_operation(posting_List1, posting_List3)
    return results,comparison

In [63]:
# This function takes the query from user and perform all the pre-processing on the query and finally returns the pre-processed query.
def EnterQuery():
  query=input('Enter your query')
  query=query.lower()
  query = re.sub('[^a-zA-Z0-9]', ' ', query)
  # query = query.replace('http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
  stop_word = stopwords.words('english')
  # query = query.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_word)]))
  query=query.split()
  query_token = [word for word in query if not word in stop_word]
  # query_token=query.apply(lemmatize_text)
  return query_token

In [64]:
# This function takes the list of operators from the user which should be one less than the query tokens and applied necessary processing
# to convert it into tokens and finally return it.
def EnterOperations(tokens):
    operations = input(f'Enter the operators list of {tokens-1} operators')
    operations = operations.replace(']','') 
    operations = operations.replace('[','')
    operations = operations.replace(' ','')
    operations = operations.lower()
    operations = operations.split(',')
    return operations
    

In [65]:
# This function takes three arguments like two posting lists and one operator and called necessary functions according to the
# value of operator and finally returns the result and number of comparisons.
def Processing_input(posting_List1, posting_List2,operator):
    if(operator == 'or'):
        result, comparison = OR_operation(posting_List1, posting_List2) 
    if(operator == 'and'): 
        result, comparison = AND_operation(posting_List1, posting_List2) 
    if(operator == 'ornot'):
        result, comparison = OR_NOT_operation(posting_List1, posting_List2)
    if(operator == 'andnot'):
        result, comparison= AND_NOT_operation(posting_List1, posting_List2) 
    return result, comparison

In [66]:
# This function takes the number of queries and perform required operations.
try:
  comparisons=0
  N=int(input('Enter the number of queries'))
  for i in range(N):
    query=EnterQuery()

    operations=EnterOperations(len(query))
    for i in range(len(operations)):
        if(i == 0):
            result = posting_list[query[0]]  
            # num_comp=num_comp+num_comp_temp  
        result,comp = Processing_input(result, posting_list[query[i+1]],operations[i])
        comparisons=comparisons+comp
    print("Number of documents retrieved:",len(result))
    print("Minimum number of comparisons done:",comparisons)
    document_names=[]
    for i in result:
      document_names.append(file_content.iloc[i,0])
    print("The list of document names retrieved: ",document_names)
except:
  print('Enter Valid Input!!!')

Enter the number of queries1
Enter your querylion stood thoughtfully for a moment
Enter the operators list of 3 operators[ OR, OR , OR ]
Number of documents retrieved: 167
Minimum number of comparisons done: 347
The list of document names retrieved:  ['anim_lif.txt', 'annoy.fascist', 'anime.lif', 'ambrose.bie', 'bbh_intv.txt', 'barney.txt', 'a_tv_t-p.com', 'b-2.jok', 'art-fart.hum', 'bmdn01.txt', 'bitnet.txt', 'beesherb.txt', 'beauty.tm', 'cabbage.txt', 'caesardr.sal', 'bw.txt', 'bw-phwan.hat', 'butwrong.hum', 'boneles2.txt', 'booze1.fun', 'christop.int', 'childhoo.jok', 'chickenheadbbs.txt', 'candy.txt', 'calculus.txt', 'consp.txt', 'conan.txt', 'computer.txt', 'collected_quotes.txt', 'cogdis.txt', 'cmu.share', 'clancy.txt', 'cybrtrsh.txt', 'coyote.txt', 'cuchy.hum', 'cookie.1', 'dthought.txt', 'dromes.txt', 'drinks.gui', 'doggun.sto', 'deep.txt', 'devils.jok', 'dead5.txt', 'dead4.txt', 'facedeth.txt', 'exam.50', 'epi_tton.txt', 'eskimo.nel', 'epi_.txt', 'epitaph', 'english.txt', 'eng